# Welcome to Justuse!

## Installation
Before we start, let's get the latest version

In [1]:
%%bash
python -m pip install justuse

In [2]:
import use

In [3]:
use.__version__

'0.5.0'

## Basic Usage

Let's start with a simple case

In [4]:
use("math").cos(23)

case = (False, False, True, False)
result = <module 'math' from '/home/thorsten/anaconda3/lib/python3.8/lib-dynload/math.cpython-38-x86_64-linux-gnu.so'>


-0.5328330203333975

Note that the line
`case = (False, False, True, False)` is only logged. 
This cryptic tuple means "no version specified, no hash given, already publicly installed and available, no auto installation requested".

Also the module/package we were able to load is logged - <module 'math' from '/home/thorsten/anaconda3/lib/python3.8/lib-dynload/math.cpython-38-x86_64-linux-gnu.so'>

Installing something could be inconvenient or unnecessary if something else is available - or we want to include some minimal functionality in our program and only fetch additional dependencies only under certain conditions.

The common approach would be something like

In [23]:
try:
    import some_big_package
except ImportError:
    some_big_package = None
if some_big_package:
    ...

which is unnecessarily cumbersome - couldn't we simply have a default like in so many other functions that is returned instead of raising an exception? Of course we can!

In [25]:
pkg = use("some_big_package", default=None)
if pkg:
    ...

case = (False, False, False, False)
result = No package installed named some_big_package and auto-installation not requested. Aborting.


Even more concise:

In [27]:
if (pkg := use("some_big_package", default=None)):
    ...

case = (False, False, False, False)
result = No package installed named some_big_package and auto-installation not requested. Aborting.


One of the most practical use()s is making sure we imported the expected version of a certain package. This is especially important in research papers, notebooks and other publications because those often don't come with a `requirements.txt` and there is no way to make sure you are actually running the published code with the same versions as the author.
Also, if you pip-install something, it can happen that it upgrades a dependency, accidentally breaking code that requires the old version - with pip you *can't have more than one version installed*. With justuse any number of versions can be installed in parallel, without interfering with anything that was installed globally via pip, conda etc.

In [8]:
np = use("numpy", version="2022")

case = (True, False, True, False)
result = numpy expected to be version 2022, but got 1.19.5 instead


AmbiguityWarning: numpy expected to be version 2022, but got 1.19.5 instead

In [9]:
np.__version__

'1.19.5'

Here you see that even though we got a warning about the wrong version, we still get the requested package, just giving you a heads up about a possibly problematic situation without standing in the way.

Let's try another one!

In [16]:
spam = use("spam")

case = (False, False, False, False)
result = No package installed named spam and auto-installation not requested. Aborting.


ImportError: No package installed named spam and auto-installation not requested. Aborting.

Well, bummer! Why can't we have spam? We're engineers and need to analyze materials! (Check it out, it's real! https://pypi.org/project/spam/ )
We need spam, give us spam!

In [18]:
spam = use("spam", modes=use.auto_install)

case = (False, False, False, True)
result = Please specify version and hash for auto-installation of 'spam'.
To get some valuable insight on the health of this package, please check out https://snyk.io/advisor/python/spam
If you want to auto-install the latest version: use("spam", version="0.5.2.1", hashes={'d6c1c1c53a988b3daf44c1865d40f86de48665639bfbd5eea1317eb083638a3a'}, modes=use.auto_install)



RuntimeWarning: Please specify version and hash for auto-installation of 'spam'.
To get some valuable insight on the health of this package, please check out https://snyk.io/advisor/python/spam
If you want to auto-install the latest version: use("spam", version="0.5.2.1", hashes={'d6c1c1c53a988b3daf44c1865d40f86de48665639bfbd5eea1317eb083638a3a'}, modes=use.auto_install)


Now we're getting somewhere. Have a look at the last line of the Error message - let's try to copy&paste it..

In [19]:
spam = use("spam", version="0.5.2.1", hashes={'d6c1c1c53a988b3daf44c1865d40f86de48665639bfbd5eea1317eb083638a3a'}, modes=use.auto_install)

case = (True, True, False, True)
calling pip to install install_item=/home/thorsten/.justuse-python/packages/spam-0.5.2.1-cp38-cp38-manylinux2014_x86_64.whl


Processing /home/thorsten/.justuse-python/packages/spam-0.5.2.1-cp38-cp38-manylinux2014_x86_64.whl
installation_path = /home/thorsten/.justuse-python/venv/spam/0.5.2.1/lib/python3.8/site-packages
module_path = /home/thorsten/.justuse-python/venv/spam/0.5.2.1/lib/python3.8/site-packages/spam/__init__.py
result = <module 'spam' from '/home/thorsten/.justuse-python/packages/spam-0.5.2.1-cp38-cp38-manylinux2014_x86_64.whl/spam/__init__.py'>


<module 'spam' from '/home/thorsten/.justuse-python/packages/spam-0.5.2.1-cp38-cp38-manylinux2014_x86_64.whl/spam/__init__.py'>

Wow, did we just download, install and load the spam package without leaving our own sweet code?? Yes, we did!

Furthermore, the package we installed is version and hash-pinned so we really only get what we asked for and nothing else.

There's a small problem though. Those hashes refer to very specific files and some of those packages may be written in C or even Fortran (like numpy) that are compiled for specific platforms.
If you happily develop code on Linux that uses something platform-specific (like numpy!) it will all work without problems - until you try to run your code on another platform. In this case, you need to specify all hashes for all platforms you want to run your code on.

## Use() modules from anywhere!
If you `import` some package or module, you're limited to the stuff you have in your current directory or below (but only if there is a `__init__.py` or if it's an implicit namespace package) and the things in your sys.path, which can be manipulated freely, making it very complicated to handle. Let's suppose we're in our test directory.

In [43]:
%cd ~/Desktop/sf_Dropbox/code/justuse/tests

/media/sf_Dropbox/code/justuse/tests


the code we want to run is in justuse/docs and there is no `__init__.py` in between, so to get to run the code, we could put the src directory in sys.path - or we could use() a module directly!

In [44]:
mod = use(use.Path("../docs/demo.py"))

In [45]:
mod.foo()

Hello justuse-user!


Loading single modules doesn't sound like much, but especially while experimenting on jupyter, this can be used very effectively in conjunction with the reloading mode:

In [37]:
mod = use(use.Path("../docs/demo.py"), modes=use.reloading)

Now this module is loaded fresh whenever you modify and save the file, replacing the implementation behind the scene. This will work without any problems as long as you put functions in that module and if you access those functions via attribute-access (`mod.func()` **not** `func = mod.func; func()`).

If you can load modules from disk, why couldn't you load them from the web? Let's say you found an interesting github repo like https://github.com/amogorkon/justuse. Chances are, there's also a package on pypi you can pip-install, but maybe there's not. Maybe you're only interested in a single module from that repo/package, so you don't even want to install anything. Then you could download it from github, move the file manually into your folder and import it - sounds like a lot of trouble for a single module!
There has to be a better way! And there is - you can just use() web resources:

In [38]:
mod = use(use.URL("https://raw.githubusercontent.com/amogorkon/justuse/unstable/docs/demo.py"))

/home/thorsten/anaconda3/lib/python3.8/site-packages/use/use.py:1636: NoValidationWarning: Attempting to import from the interwebs with no validation whatsoever!
To safely reproduce: use(use.URL('https://raw.githubusercontent.com/amogorkon/justuse/unstable/docs/demo.py'), hash_algo=use.Hash.sha256, hash_value='59eff31bb220ce933ccc083b9306020ec25d19d43de30e5ad4341b355d4b48bf')
  warn(Message.no_validation(url, hash_algo, this_hash), NoValidationWarning)


copy&paste that line from the exception to get that sweet hash..

In [40]:
mod = use(use.URL('https://raw.githubusercontent.com/amogorkon/justuse/unstable/docs/demo.py'), hash_algo=use.Hash.sha256, hash_value='59eff31bb220ce933ccc083b9306020ec25d19d43de30e5ad4341b355d4b48bf')

In [41]:
mod.foo()

Hello justuse-user!


Since the content of this file is now hash-pinned, it doesn't matter whether or not someone hacks github and changes the code - justuse will instantly notice before executing any code. You can even execute code directly from pastebin or any other untrusted, public platform - as long as you have the proper hash, you're safe.